In [1]:
import os
import numpy as np
import sys
import time
import _init_paths
from core.config import cfg, merge_cfg_from_file, merge_cfg_from_list, assert_and_infer_cfg
from core.test_engine import run_inference, get_inference_dataset, extend_results
from datasets.json_dataset import JsonDataset
import pickle
import json
import ipdb
from pycocotools.cocoeval import COCOeval
from datasets.json_dataset_evaluator import _log_detection_eval_metrics, _coco_bbox_results_one_category

In [2]:
def box_equal(box1, box2):
    for i in range(len(box1)):
        if abs(box1[i] - box2[i]) >= 0.01:
            return False
    return True

def dict_equal(dict1, dict2):
    a =  dict1['image_id'] == dict2['image_id']
    b = dict1['category_id'] == dict2['category_id']
    c = dict1['score'] == dict2['score']
    d  = box_equal(dict1['bbox'], dict2['bbox'])
    return a&b&c&d

In [2]:
res_file = '../Outputs/e2e_faster_rcnn_R-50-FPN_1x_vcoco/Nov26-09-29-09_wanbo_node35_step/test/bbox_vcoco_test_results.json'
det_res = json.load(open(res_file, 'rb'))
# res_file = '../Outputs/e2e_faster_rcnn_R-50-FPN_1x_vcoco/Nov26-09-29-09_wanbo_node35_step/test/bbox_vcoco_trainval_results.json'
# det_res = json.load(open(res_file, 'rb'))

In [20]:
det_results = {}
for det in det_res:
    im_id = det['image_id']
    tmp = det_results.setdefault(im_id, [])
    tmp.append(np.concatenate([det['bbox'], [det['score'], det['category_id']]]))
    det_results[im_id] = tmp

In [21]:
from utils.boxes import xywh_to_xyxy, bbox_overlaps
for k,v in det_results.items():
    v = np.array(v).astype(np.float32)
    trans = xywh_to_xyxy(v[:, :4])
    v[:, :4] = trans
    det_results[k] = v

In [22]:
save_path = '../data/cache/vcoco_test_precomp_boxes.json'
# save_path = '../data/cache/vcoco_trainval_precomp_boxes.json'
for k,v in det_results.items():
    det_results[k] = v.tolist()
    
with open(save_path, 'w') as f:
    json.dump(det_results, f)

In [5]:
cfg.TEST.DATASETS = ('vcoco_test',)
cfg.MODEL.NUM_CLASSES = 81
cfg.TEST.PRECOMPUTED_PROPOSALS = False
cfg.MODEL.VCOCO_ON = True
cfg.VCOCO.USE_PRECOMP_BOX = True
dataset_name, proposal_file = get_inference_dataset(0, is_parent=False)
json_dataset = JsonDataset(dataset_name)
vcocodb = json_dataset.get_roidb(gt=True)

loading annotations into memory...
Done (t=1.22s)
creating index...
index created!
loading vcoco annotations...
add precomp box from /root/project/InteractNet/data/cache/vcoco_test_precomp_boxes.json


In [6]:
ican_file = '../ican/Test_Faster_RCNN_R-50-PFN_2x_VCOCO.pkl'
# open(ican_file, 'rb')
ican = pickle.load(open(ican_file, 'rb'), encoding='latin1')

In [17]:
from utils.boxes import xywh_to_xyxy, xyxy_to_xywh, bbox_overlaps
ican_det_res = []
for im_id, v in ican.items():
    for item in v:
        res = {}
        res['image_id'] = im_id
        res['category_id'] = item[-2]
        box = xyxy_to_xywh(item[2][None, :])[0]
        res['bbox'] = box.tolist()
#         res['bbox'] = [box[0]*scale[1], box[1]*scale[0], box[2]*scale[1], box[3]*scale[0]]
        res['score'] = item[-1].tolist()
        ican_det_res.append(res)

In [15]:
xyxy_to_xywh(ican[294][0][2][None, :])[0].tolist()
# np.max(ican[294][0][5]) > 0

[24.310546875, 65.66578674316406, 349.72113037109375, 350.07659912109375]

In [18]:
len(ican_det_res)

119061

In [23]:
print(ican_det_res[100])
print(det_res[0])
ican_res_file = '../ican/ican_vcoco_det_res.json'
with open(ican_res_file, 'w') as f:
    json.dump(ican_det_res, f)

{'image_id': 385029, 'category_id': 51, 'bbox': [217.9744873046875, 170.88516235351562, 28.596038818359375, 30.3131103515625], 'score': 0.06117446720600128}
{'image_id': 294, 'category_id': 1, 'bbox': [20.025070190429688, 60.60774230957031, 344.08143615722656, 360.11997985839844], 'score': 0.9992368221282959}


In [20]:
coco_dt = json_dataset.COCO.loadRes(ican_res_file)
coco_eval = COCOeval(json_dataset.COCO, coco_dt, 'bbox')
coco_eval.evaluate()
coco_eval.accumulate()
_log_detection_eval_metrics(json_dataset, coco_eval)

Loading and preparing results...
DONE (t=1.46s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=25.72s).
Accumulating evaluation results...
DONE (t=4.84s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.065
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.102
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.072
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.043
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.072
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.096
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.057
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.088
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.090
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1

In [10]:
cfg.TEST.DATASETS = ('vcoco_test',)
cfg.MODEL.NUM_CLASSES = 81
cfg.TEST.PRECOMPUTED_PROPOSALS = False
cfg.MODEL.VCOCO_ON = True
cfg.VCOCO.USE_PRECOMP_BOX = True
dataset_name, proposal_file = get_inference_dataset(0, is_parent=False)
json_dataset = JsonDataset(dataset_name)
vcocodb = json_dataset.get_roidb(gt=True)

loading annotations into memory...
Done (t=0.94s)
creating index...
index created!
loading vcoco annotations...
add precomp box from /root/project/InteractNet/data/cache/vcoco_test_precomp_boxes.json


In [12]:
from core.test import im_detect_all, im_detect_bbox_precomp_box
num_classes = cfg.MODEL.NUM_CLASSES
num_images = len(vcocodb)
all_boxes = [[[] for _ in range(num_images)] for _ in range(num_classes)]
for i, entry in enumerate(vcocodb):
    scores, boxes, cls_boxes_i = im_detect_bbox_precomp_box(entry, json_dataset.json_category_id_to_contiguous_id)
    extend_results(i, all_boxes, cls_boxes_i)

In [13]:
results = []
for cls_ind, cls in enumerate(json_dataset.classes):
    if cls == '__background__':
        continue
    if cls_ind >= len(all_boxes):
        break
    cat_id = json_dataset.category_to_id_map[cls]
    results.extend(_coco_bbox_results_one_category(
        json_dataset, all_boxes[cls_ind], cat_id))

In [16]:
len(results)
len(det_res)

156926

In [17]:
count = 0
for i in range(len(results)):
    if not dict_equal(results[i], det_res[i]):
        count += 1
        print(results[i], filter_det_res[i])
        ipdb.set_trace()
#         break
print(count)

0


In [18]:
from datasets.task_evaluation import json_dataset_evaluator
coco_eval = json_dataset_evaluator.evaluate_boxes(
    json_dataset, all_boxes, './', use_salt=False, cleanup=False)

Loading and preparing results...
DONE (t=0.95s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=38.82s).
Accumulating evaluation results...
DONE (t=5.76s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.288
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.486
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.302
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.177
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.337
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.422
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.271
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.407
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.422
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1

In [4]:
import pickle
res_file = '../Outputs/e2e_interact_net_R-50-FPN_1x/Dec30-10-43-23_wanbo-node33_step/test/detections.pkl'
det_res = pickle.load(open(res_file, 'rb'))

In [8]:
sum(det_res['all_losses']['affinity_pos_num'])

6681

In [9]:
sum(det_res['all_losses']['affinity_tp_num'])

4666

In [10]:
4666/6681

0.6983984433468043

In [11]:
sum(det_res['all_losses']['affinity_neg_num'])

358788

In [12]:
sum(det_res['all_losses']['affinity_tf_num'])

355983

In [13]:
355983/358788

0.992182012776347